In [3]:
import torch

model_name_or_path = "THUDM/chatglm3-6b"
train_data_path = (
    "data/zhouyi_dataset_20240302_180153.csv"  # 训练数据路径(批量生成数据集）
)
eval_data_path = None  # 验证数据路径，如果没有则设置为None
seed = 8  # 随机种子
max_input_length = 512  # 输入的最大长度
max_output_length = 1536  # 输出的最大长度
lora_rank = 16  # LoRA秩
lora_alpha = 32  # LoRA alpha值
lora_dropout = 0.05  # LoRA Dropout率
prompt_text = ""  # 所有数据前的指令文本

In [ ]:
from datasets import load_dataset, ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML
from transformers import AutoTokenizer

dataset = load_dataset("csv", data_files=train_data_path)


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))


show_random_elements(dataset["train"], num_examples=5)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, trust_remote_code=True, revision="b098244"
)